# Задача 3. Фибоначчи-2


### Алгоритм возведения в степень возведением в квадрат и умножением для вычисления чисел Фибоначчи через матрицу Фибоначчи (с использованием остатка от деления)

Источник (адаптированный для матриц): https://habr.com/ru/post/344666/

![pircture](https://drive.google.com/uc?export=view&id=1m3jPuW_vIL-jRFLA7K3ICuWPLSkmTrJX)


В этом примере не показано, что при умножении матриц используются остатки от деления на заданное число ${M}$, когда цифры в матрицах становятся больше этого числа ${M}$. Кроме того, возвращается не само число Фибоначчи, а остаток от его деления на число ${M}$.

In [ ]:
import numpy as np

In [ ]:
# Эта функция вычисляет левую колонку чисел из примера в начале, т. е. частные от деления
# степени нужной мне матрицы Фиб. на 2 до тех пор, пока частное не станет единицей
def quotients(p: int):
    """Return a list of quotients of power p divided successively in half.

    Example: quotients(5) -> [5, 2, 1]
    """
    if p <= 1:
        raise ValueError('power must be more than 1')
    p_quotients = [p]
    # сколько раз можно разделить p на 2? Ответ: log2(p)
    n = np.floor(np.log2(p)).astype(int)
    for i in range(n):
        p_quotient = p//2
        p_quotients.append(p_quotient)
        p = p_quotient
    return p_quotients

In [ ]:
#  Эта функция вычисляет правую колонку матриц из примера в начале
def pows_fib_matrices(n, m=None, verbose=False):
    """Return n Fib matrices successively raised to the power of 2.

    m : if provided, use remainders of division by m instead of actual matrices

    return : a list of numpy arrays (2x2)

    Example: pows_fib_matrices(3) - > [array([[1, 1], [1, 0]]),
                                       array([[2, 1], [1, 1]]),
                                       array([[5, 3], [3, 2]]) ]
    """
    fib_matrix = np.array([[1, 1], [1, 0]])
    fib_matrices = [fib_matrix]
    count_mult = 0
    for i in range(n - 1):  # т.к. начальная матр. Фиб. уже в списке
        if m is not None and fib_matrix[1, 1] > m:
            fib_matrix = fib_matrix % m
        fib_matrix_pow2 = fib_matrix.dot(fib_matrix)
        fib_matrices.append(fib_matrix_pow2)
        fib_matrix = fib_matrix_pow2
        count_mult += 1
        if verbose == True:
            print('Matrix mult count:', count_mult)
        if (fib_matrix < 0).any():
                raise OverflowError('Neg matrix! Use m parameter', fib_matrix)

    return fib_matrices

In [ ]:
# Эта функция перемножает те матрицы из правой колонки из примера в начале, которым
# соответствуют нечетные числа в левой колонке 
def product_fib_matrices(p_quotients, fib_matrices, m=None, verbose=False):
    """Return the product of fib_matrices corresponding to odd p_quotients.
    
    m : if provided, use remainders of division by m instead of actual matrices
    return : numpy array dim (2, 2)

    Example: 
    p_quotients = [5, 2, 1]
    fib_matrices = [array([[1, 1], [1, 0]]),
                    array([[2, 1], [1, 1]]),
                    array([[5, 3], [3, 2]])]
    product_fib_matrices(p_quotients, fib_matrices) -> array([[8, 5], [5, 3]])
    """
    fib_matrix = np.eye(2).astype('int64')
    count_mult = 0
    for idx, val in enumerate(p_quotients):
        if val % 2 != 0:
            if m is not None and fib_matrix[1, 1] > m:
                fib_matrix = fib_matrix % m
            fib_matrix = fib_matrix.dot(fib_matrices[idx])
            count_mult += 1
            if verbose == True:
                print('Matrix mult count:', count_mult)
            if (fib_matrix < 0).any():
                raise OverflowError('Neg matrix! Use m parameter', fib_matrix)

    return fib_matrix

Результат: Для решения этой задачи напишите программу, которая принимает на вход два числа n и m и выводит остаток от деления n-го (нумерация чисел в последовательности начинается с 1) числа Фибоначчи на число m.

Пример входных данных:

1000000000001 99999

Пример результата:

63715 


In [ ]:
# онлайн-калькулятор чисел Фиб.: https://www.omnicalculator.com/math/fibonacci

# Функция-интерфейс для использования кода
def fib_remainder(nth_fib, m=None, verbose=False):
    """Return the remainder of division of the nth Fib number by m.
    
    If m is not provided, returns the nth Fib number but up to about 90 only,
    and raises OverflowError when it cannot calculate the number correctly.

    verbose : when True, print the count of matrix multiplications performed 
    """
    # считаем левую колонку цифр из алгоритма, показанного в начале
    # (nth_fib-2) т.к., напр., для 7-го числа Ф. (13) нужна матрица Ф. в 5-ой степени 
    p_quotients = quotients(nth_fib-2)
    # считаем правую колонку матриц из алгоритма, показанного в начале
    fib_matrices = pows_fib_matrices(len(p_quotients), m, verbose)
    # произведение матриц из правой колонки, соответствующих нечетным цифрам из левой
    prod_fib_mat = product_fib_matrices(p_quotients, fib_matrices, m, verbose)

    # считаем остаток от деления числа Фиб. на m либо считаем само число Фиб.
    if m is not None:
        fib = prod_fib_mat.dot(np.array([1, 1]))[0] % m
    else:
        fib = prod_fib_mat.dot(np.array([1, 1]))[0]
    
    return fib

In [ ]:
assert fib_remainder(nth_fib=1000000000001, m=99999) == 63715

In [ ]:
# Можно посмотреть, сколько матричных умножений происходит при расчете результата

# assert fib_remainder(nth_fib=1000000000001, m=99999, verbose=True) == 63715

In [ ]:
# Может считать числа Фиб. без использования остатка (параметра m) где-то до 90 числа
# Потом вызывает ошибку OverflowError

# F50=12586269025
# assert fib_remainder(nth_fib=50) == 12586269025

# fib_remainder(nth_fib=95)  # Вызывает ошибку

In [ ]:
import unittest


class TestFib(unittest.TestCase):
    """Some tests to check that changes to my code do not break it."""

    def test_quotients(self):
        """Test that quotients function returns the right list."""
        param_list = [(4, [4, 2, 1]),
                      (13, [13, 6, 3, 1])
        ]
        for power, expected_quotients_list in param_list:
            with self.subTest():
              self.assertEqual(quotients(power), expected_quotients_list)

    def test_quotients_last_el_one(self):
        self.assertEqual(quotients(331)[-1], 1, 'last element must be 1')
    
    def test_quotients_raises_error(self):
        """If the p (power) parameter is <= 1, error must be raised."""
        with self.assertRaises(ValueError):
            quotients(1)

    def test_pows_fib_matrices(self):
        fib_pow5 = np.array(
            [
             [[1, 1],
              [1, 0]], 
             [[2, 1],
              [1, 1]],
             [[5, 3],
              [3, 2]]
             ]
             )
        self.assertTrue((pows_fib_matrices(3) == fib_pow5).all())

    def test_pows_fib_matrices_raises_error(self):
          """Must raise error if no m is given and the Fib num is too large."""
          with self.assertRaises(OverflowError):
              pows_fib_matrices(110)
        
    def test_product_fib_matrices(self):
        p_quotients = [4, 2, 1]
        fib_matrices = pows_fib_matrices(len(p_quotients))
        expected = np.array([[5, 3], [3, 2]])
        self.assertTrue((product_fib_matrices(p_quotients, fib_matrices) == expected).all())

        p_quotients = [5, 2, 1]
        fib_matrices = pows_fib_matrices(len(p_quotients))
        expected = np.array([[8, 5], [5, 3]])
        self.assertTrue((product_fib_matrices(p_quotients, fib_matrices) == expected).all())

        p_quotients = quotients(1000000000001)
        fib_matrices = pows_fib_matrices(len(p_quotients), m=99999)
        m = 'All numbers must be positive'
        self.assertTrue(product_fib_matrices(p_quotients, fib_matrices, m=99999).all()  > 0, m)

    def test_product_fib_matrices_raises_error(self):
          """Must raise error if no m is given and the Fib num is too large."""
          p_quotients = quotients(1000000000001)
          fib_matrices = pows_fib_matrices(len(p_quotients), m=99999)
          with self.assertRaises(OverflowError):
              product_fib_matrices(p_quotients, fib_matrices)
      
    def test_fib_remainder_no_m_given_fib_small(self):
        """fib_remainder can compute small Fib numbers."""
        # F50=12586269025
        self.assertEqual(fib_remainder(nth_fib=50), 12586269025)

    def test_fib_remainder_no_m_given_raises_error_fib_large(self):
          """Must raise error if no m is given and the Fib num is too large."""
          with self.assertRaises(OverflowError):
              fib_remainder(nth_fib=110)

    def test_fib_remainder_m_given(self):
        # F7 = 13, 13%5 = 3
        # F20 = 6765, 6765%7 = 3
        # F100 = 354224848179261915075; % 999 = 858
        # F100 % 99999 = 46740
        # F110 = 43566776258854844738105
        # F129 = 407305795904080553832073954
        param_list = [(7, 5, 3), (20, 9, 6), (100, 999, 858),
                      (100, 99999, 46740), (110, 999, 188),
                      (110, 9999, 4499), (129, 99999, 89332),
                      (1000000000001, 99999, 63715)
                      ]
        for nth_fib, m, expected in param_list:
            with self.subTest():
                self.assertEqual(fib_remainder(nth_fib=nth_fib, m=m), expected)
        
    
tests = TestFib()

tests_loaded = unittest.TestLoader().loadTestsFromModule(tests)

unittest.TextTestRunner(verbosity=2).run(tests_loaded)

test_fib_remainder_m_given (__main__.TestFib) ... ok
test_fib_remainder_no_m_given_fib_small (__main__.TestFib)
fib_remainder can compute small Fib numbers. ... ok
test_fib_remainder_no_m_given_raises_error_fib_large (__main__.TestFib)
Must raise error if no m is given and the Fib num is too large. ... ok
test_pows_fib_matrices (__main__.TestFib) ... ok
test_pows_fib_matrices_raises_error (__main__.TestFib)
Must raise error if no m is given and the Fib num is too large. ... ok
test_product_fib_matrices (__main__.TestFib) ... ok
test_product_fib_matrices_raises_error (__main__.TestFib)
Must raise error if no m is given and the Fib num is too large. ... ok
test_quotients (__main__.TestFib)
Test that quotients function returns the right list. ... ok
test_quotients_last_el_one (__main__.TestFib) ... ok
test_quotients_raises_error (__main__.TestFib)
If the p (power) parameter is <= 1, error must be raised. ... ok

----------------------------------------------------------------------
Ran 10

<unittest.runner.TextTestResult run=10 errors=0 failures=0>